In [366]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
%matplotlib inline



    Прочитать про методы оптимизации для нейронных сетей https://habr.com/post/318970/

    Реализовать самостоятельно логистическую регрессию
        Обучить ее методом градиентного спуска
        Методом nesterov momentum
        Методом rmsprop

    В качестве dataset’а взять Iris, оставив 2 класса:
        Iris Versicolor
        Iris Virginica



In [7]:
# Загружаем набор данных Ирисы:
iris = datasets.load_iris()
# Смотрим на названия переменных
print(iris.feature_names)
# Смотрим на данные, выводим 10 первых строк: 
print(iris.data[:10])
# Смотрим на целевую переменную:
print (iris.target_names)
print (iris.target)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]]
['setosa' 'versicolor' 'virginica']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [16]:
iris_frame = pd.DataFrame(iris.data)
# Делаем имена колонок такие же, как имена переменных:
iris_frame.columns = iris.feature_names
# Добавляем столбец с целевой переменной: 
iris_frame['target'] = iris.target
# Для наглядности добавляем столбец с сортами: 
iris_frame['name'] = iris_frame.target.apply(lambda x : iris.target_names[x])
# Смотрим, что получилось:
iris_frame.head(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [29]:
data=iris_frame[iris_frame['target']==1].append(iris_frame[iris_frame['target']==2])

In [31]:
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,name
50,7.0,3.2,4.7,1.4,1,versicolor
51,6.4,3.2,4.5,1.5,1,versicolor
52,6.9,3.1,4.9,1.5,1,versicolor
53,5.5,2.3,4.0,1.3,1,versicolor
54,6.5,2.8,4.6,1.5,1,versicolor


Чтобы перейти к логистической регрессии, target=1 заменим на -1

In [84]:
data['target'][data['target']==2]=-1

/home/benito/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Разделим выборку на X и y

In [96]:
X=data[['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']]

In [97]:
y=data[['target']]

In [154]:
def logistic_regression(x, b): # функция логистической регрессии
    x_new = np.array([0])
    x_new=np.concatenate((x_new, x))
    z=np.dot(x_new,b)
    y=np.exp(z)/(1+np.exp(z))
    if y<=0.5:
        return(-1)
    else:
        return(1)

In [209]:
def log_loss(X,y,b0,b1,b2,b3,b4): # логистическая функция потерь
    b = np.array([b0,b1,b2,b3,b4])
    log_loss=0
    for i in range(len(X)):
        x=np.array(X.iloc[i])
        x_new = np.array([0])
        x_new=np.concatenate((x_new, x))
        z=np.dot(x_new,b)
        log_loss=log_loss+np.log(1+np.exp(-y.iloc[i]*z))
    return log_loss

Необходимо минимизировать функцию log_loss

### 1 пункт

Метод градиентного спуска. Алгоритм:
1. Задаем $\gamma$ - "learning rate"
2. Выбираем начальное приближение $x_0$
3. for k = 0, 1, 2 ... do
    1. $s_k$ = -$\nabla f(x_k)$
    2. $x_{k+1} = x_k + \gamma s_k$

In [211]:
def target_log_loss(b0,b1,b2,b3,b4): # функция для наших данных
    return log_loss(X,y,b0,b1,b2,b3,b4)

In [239]:
def grad(b0,b1,b2,b3,b4,delta): # функция для градиента
    df0=(target_log_loss(b0+delta,b1,b2,b3,b4)-target_log_loss(b0-delta,b1,b2,b3,b4))/(2*delta)
    df1=(target_log_loss(b0,b1+delta,b2,b3,b4)-target_log_loss(b0,b1-delta,b2,b3,b4))/(2*delta)
    df2=(target_log_loss(b0,b1,b2+delta,b3,b4)-target_log_loss(b0,b1,b2-delta,b3,b4))/(2*delta)
    df3=(target_log_loss(b0,b1,b2,b3+delta,b4)-target_log_loss(b0,b1,b2,b3-delta,b4))/(2*delta)
    df4=(target_log_loss(b0,b1,b2,b3,b4+delta)-target_log_loss(b0,b1,b2,b3,b4-delta))/(2*delta)
    return df0,df1,df2,df3,df4

In [277]:
def grad_desc(b0,b1,b2,b3,b4,delta,step,it): # функция для градиентного спуска
    for i in range(it):
        b0=b0-step*grad(b0,b1,b2,b3,b4,delta)[0]
        b1=b1-step*grad(b0,b1,b2,b3,b4,delta)[1]
        b2=b2-step*grad(b0,b1,b2,b3,b4,delta)[2]
        b3=b3-step*grad(b0,b1,b2,b3,b4,delta)[3]
        b4=b4-step*grad(b0,b1,b2,b3,b4,delta)[4]
    return b0,b1,b2,b3,b4

In [369]:
b0=1
b1=1
b2=1
b3=1
b4=1
delta=1
step=0.0008
it=5
b=grad_desc(b0,b1,b2,b3,b4,delta,step,it)

In [370]:
target_log_loss(b[0],b[1],b[2],b[3],b[4])

target    109.799069
dtype: float64

### 2 пункт

Метод nesterov momentum
Алгоритм:
- считаем старый градиент
- считаем промежуточный градиент в точке старых весов - 0.9 старого градиента
- новые веса будут равны старым весам - взвешанная сумма старого градиента и промежуточного

In [279]:
def grad_desc_nm(b0,b1,b2,b3,b4,delta,step,it,gamma): # функция для градиентного спуска c моментом нестерова
    grad_old=grad(b0,b1,b2,b3,b4,delta) # страый градиент
    for i in range(it):
        grad_curr=grad(b0,b1,b2,b3,b4,delta) # обновляем текущий градиент
        b0_inter=b0-step*gamma*grad_old[0]
        b1_inter=b1-step*gamma*grad_old[1]
        b2_inter=b2-step*gamma*grad_old[2]
        b3_inter=b3-step*gamma*grad_old[3]
        b4_inter=b4-step*gamma*grad_old[4]
        grad_inter=grad(b0_inter,b1_inter,b2_inter,b3_inter,b4_inter,delta) # обновляем промежутчный градиент
        b0=b0-step*((1-gamma)*grad_inter[0]+gamma*grad_old[0]) # обновляем параметры
        b1=b1-step*((1-gamma)*grad_inter[1]+gamma*grad_old[1]) # старые точки минус
        b2=b2-step*((1-gamma)*grad_inter[2]+gamma*grad_old[2]) # шаг умноженные на взвешанную сумму
        b3=b3-step*((1-gamma)*grad_inter[3]+gamma*grad_old[3]) # старого градиента и промежутночного
        b4=b4-step*((1-gamma)*grad_inter[4]+gamma*grad_old[4])
        old_curr=grad_curr # обновляем старый градиент
    return b0,b1,b2,b3,b4

In [374]:
b0=1
b1=1
b2=1
b3=1
b4=1
delta=1
step=0.0008
it=5
gamma=0.9
b=grad_desc_nm(b0,b1,b2,b3,b4,delta,step,it,gamma)

In [376]:
target_log_loss(b[0],b[1],b[2],b[3],b[4])

target    75.600834
dtype: float64

### 3 пункт

Методом rmsprop.
Алгоритм:
 - считаем взвешанную сумму квадратов градиентов
 - к старым параметрам градиент с шагом step/корень из взвешанной суммы квадратов

In [392]:
def grad_desc_rmspop(b0,b1,b2,b3,b4,delta,step,it,gamma): # функция для градиентного спуска rmspop
    e_0=0
    e_1=0
    e_2=0
    e_3=0
    e_4=0
    for i in range(it):
        grad_curr=grad(b0,b1,b2,b3,b4,delta)
        e_0=gamma*e_0+(1-gamma)*grad_curr[0]*grad_curr[0]
        e_1=gamma*e_1+(1-gamma)*grad_curr[1]*grad_curr[1]
        e_2=gamma*e_2+(1-gamma)*grad_curr[2]*grad_curr[2]
        e_3=gamma*e_3+(1-gamma)*grad_curr[3]*grad_curr[3]
        e_4=gamma*e_4+(1-gamma)*grad_curr[4]*grad_curr[4]
        b0=b0-(step/np.sqrt(e_0+0.001))*grad_curr[0]
        b1=b1-(step/np.sqrt(e_1+0.001))*grad_curr[1]
        b2=b2-(step/np.sqrt(e_2+0.001))*grad_curr[2]
        b3=b3-(step/np.sqrt(e_3+0.001))*grad_curr[3]
        b4=b4-(step/np.sqrt(e_4+0.001))*grad_curr[4]
    return b0,b1,b2,b3,b4

In [417]:
b0=1
b1=1
b2=1
b3=1
b4=1
delta=1
step=1
it=5
gamma=0.9
b=grad_desc_rmspop(b0,b1,b2,b3,b4,delta,step,it,gamma)

In [418]:
target_log_loss(b[0],b[1],b[2],b[3],b[4])

target    61.18735
dtype: float64